# Logistic regression example
### Dr. Tirthajyoti Sarkar, Fremont, CA 94536

---

This notebook demonstrates solving a logistic regression problem of predicting Hypothyrodism with **Scikit-learn** and **Statsmodels** libraries.

The dataset is taken from UCI ML repository.
<br>Here is the link: https://archive.ics.uci.edu/ml/datasets/Thyroid+Disease

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Read the dataset

In [2]:
names = 'response age sex on_thyroxine query_on_thyroxine antithyroid_medication thyroid_surgery query_hypothyroid query_hyperthyroid pregnant \
sick tumor lithium goitre TSH_measured TSH T3_measured \
T3 TT4_measured TT4 T4U_measured T4U FTI_measured FTI TBG_measured TBG'

In [3]:
names = names.split(' ')

In [4]:
!wget https://raw.githubusercontent.com/tirthajyoti/Machine-Learning-with-Python/master/Datasets/hypothyroid.csv

!mkdir Data
!mv hypothyroid.csv Data/

'wget' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
df = pd.read_csv('Data/hypothyroid.csv',index_col=False,names=names,na_values=['?'])

FileNotFoundError: File b'Data/hypothyroid.csv' does not exist

In [ ]:
df.head()

In [ ]:
to_drop=[]
for c in df.columns:
    if 'measured' in c or 'query' in c:
        to_drop.append(c)

In [ ]:
to_drop

In [ ]:
to_drop.append('TBG')

In [ ]:
df.drop(to_drop,axis=1,inplace=True)

In [ ]:
df.head()

### Let us see the basic statistics on the dataset

In [ ]:
df.describe().T

### Are any data points are missing? We can check it using `df.isna()` method
The `df.isna()` method gives back a full DataFrame with Boolean values - True for data present, False for missing data. We can use `sum()` on that DataFrame to see and calculate the number of missing values per column.

In [ ]:
df.isna().sum()

### We can use `df.dropna()` method to drop those missing rows

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

### Creating a transformation function to convert `+` or `-` responses to 1 and 0

In [ ]:
def class_convert(response):
    if response=='hypothyroid':
        return 1
    else:
        return 0

In [ ]:
df['response']=df['response'].apply(class_convert)

In [ ]:
df.head()

In [ ]:
df.columns

### Exploratory data analysis

In [ ]:
for var in ['age','TSH','T3','TT4','T4U','FTI']:
    sns.boxplot(x='response',y=var,data=df)
    plt.show()

In [ ]:
sns.pairplot(data=df[df.columns[1:]],diag_kws={'edgecolor':'k','bins':25},plot_kws={'edgecolor':'k'})
plt.show()

### Create dummy variables for the categorical variables

In [ ]:
df_dummies = pd.get_dummies(data=df)

In [ ]:
df_dummies.shape

In [ ]:
df_dummies.sample(10)

### Test/train split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_dummies.drop('response',axis=1), 
                                                    df_dummies['response'], test_size=0.30, 
                                                    random_state=42)

In [ ]:
print("Training set shape",X_train.shape)
print("Test set shape",X_test.shape)

### Using `LogisticRegression` estimator from Scikit-learn
We are using the L2 regularization by default

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf1 = LogisticRegression(penalty='l2',solver='newton-cg')

In [ ]:
clf1.fit(X_train,y_train)

### Intercept, coefficients, and score

In [ ]:
clf1.intercept_

In [ ]:
clf1.coef_

In [ ]:
clf1.score(X_test,y_test)

### For `LogisticRegression` estimator, there is a special `predict_proba` method which computes the raw probability values

In [ ]:
prob_threshold = 0.5

In [ ]:
prob_df=pd.DataFrame(clf1.predict_proba(X_test[:10]),columns=['Prob of NO','Prob of YES'])
prob_df['Decision']=(prob_df['Prob of YES']>prob_threshold).apply(int)
prob_df

In [ ]:
y_test[:10]

### Classification report, and confusion matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_test, clf1.predict(X_test)))

In [ ]:
pd.DataFrame(confusion_matrix(y_test, clf1.predict(X_test)),columns=['Predict-YES','Predict-NO'],index=['YES','NO'])

### Using `statsmodels` library

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
df_dummies.columns

### Create a 'formula' in the same style as in R language

In [ ]:
formula = 'response ~ ' + '+'.join(df_dummies.columns[1:])

In [ ]:
formula

### Fit a GLM (Generalized Linear model) with this formula and choosing `Binomial` as the family of function

In [ ]:
model = smf.glm(formula = formula, data=df_dummies, family=sm.families.Binomial())

In [ ]:
result=model.fit()

### `summary` method shows a R-style table with all kind of statistical information

In [ ]:
print(result.summary())

### The `predict` method computes probability for the test dataset

In [ ]:
result.predict(X_test[:10])

### To create binary predictions, you have to apply a threshold probability and convert the booleans into integers

In [ ]:
y_pred=(result.predict(X_test)>prob_threshold).apply(int)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
pd.DataFrame(confusion_matrix(y_test, y_pred),columns=['Predict-YES','Predict-NO'],index=['YES','NO'])

### A smaller model with only first few variables

We saw that majority of variables in the logistic regression model has p-values very high and therefore they are not statistically significant. We create another smaller model removing those variables.

In [ ]:
formula = 'response ~ ' + '+'.join(df_dummies.columns[1:7])
formula

In [ ]:
model = smf.glm(formula = formula, data=df_dummies, family=sm.families.Binomial())
result=model.fit()
print(result.summary())

In [ ]:
y_pred=(result.predict(X_test)>prob_threshold).apply(int)
print(classification_report(y_pred,y_test))

In [ ]:
pd.DataFrame(confusion_matrix(y_test, y_pred),columns=['Predict-YES','Predict-NO'],index=['YES','NO'])

### How do the probabilities compare between `Scikit-learn` and `Statsmodels` predictions? 

In [ ]:
sklearn_prob = clf1.predict_proba(X_test)[...,1][:10]
statsmodels_prob =  result.predict(X_test[:10])

In [ ]:
prob_comp_df=pd.DataFrame(data={'Scikit-learn Prob':list(sklearn_prob),'Statsmodels Prob':list(statsmodels_prob)})
prob_comp_df

### Coefficient interpretation

What is the interpretation of the coefficient value for `age` and `FTI`?

- With every one year of age increase, the log odds of the hypothyrodism **increases** by 0.0248 or the odds of hypothyroidsm increases by a factor of exp(0.0248) = 1.025 i.e. almost 2.5%.
- With every one unit of FTI increase, the log odds of the hypothyrodism **decreases** by 0.1307 or the odds of hypothyroidsm decreases by a factor of exp(0.1307) = 1.1396 i.e. almost by 12.25%.